# LAB: Feature Selection

En este lab vamos a explorar selección de características. Seguiremos trabajando sobre el dataset de Titanic.

Ante que nada, carguemos algunas cosas:

- Paquetes estándar
- El dataset

In [27]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('../Data/train.csv')

In [28]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1. Preprocesamiento

Va a ser necesario realizar algunas tareas de preprocesamiento para este dataset:

1. Estandarizar la edad (`age`) y la tarifa (`fare`)
2. Construir algunas variables dummies para algunas variables relevantes (tales como `P_class`, `Embarked`, `Male`)

In [29]:
from sklearn.preprocessing import scale

In [30]:
df[['Age']].describe()

,Age
count,714.000000
mean,29.699118
std,14.526497
min,0.420000
25%,20.125000
50%,28.000000
75%,38.000000
max,80.000000


In [31]:
df['Age'].fillna((df['Age'].mean()), inplace=True)

In [32]:
df['Fare'].fillna((df['Fare'].mean()), inplace=True)

In [33]:
df['Age'] = scale(df['Age'])
df['Fare'] = scale(df['Fare'])

In [34]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,-0.592481,1,0,A/5 21171,-0.502445,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.638789,1,0,PC 17599,0.786845,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,-0.284663,0,0,STON/O2. 3101282,-0.488854,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.407926,1,0,113803,0.420730,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,0.407926,0,0,373450,-0.486337,NaN,S


In [35]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)

In [36]:
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",-0.592481,1,0,A/5 21171,-0.502445,NaN,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.638789,1,0,PC 17599,0.786845,C85,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",-0.284663,0,0,STON/O2. 3101282,-0.488854,NaN,0,0,1,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.407926,1,0,113803,0.420730,C123,0,0,1,0,0
4,5,0,"Allen, Mr. William Henry",0.407926,0,0,373450,-0.486337,NaN,1,0,1,0,1


In [37]:
df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [38]:
df.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0,-0.592481,1,0,-0.502445,1,0,1,0,1
1,1,0.638789,1,0,0.786845,0,0,0,0,0
2,1,-0.284663,0,0,-0.488854,0,0,1,0,1
3,1,0.407926,1,0,0.420730,0,0,1,0,0
4,0,0.407926,0,0,-0.486337,1,0,1,0,1


## 2. Feature selection

Utilicemos el método "SelectKBest" de scikit learn a ver cuáles son las top 5 características.

Cuáles son?


Guardémoslas en una variable llamada "kbest_columns"

In [40]:
from sklearn.feature_selection import SelectKBest, f_classif

feat_selector = SelectKBest(f_classif, k='all')
feat_selector.fit(df.drop('Survived', axis=1), df['Survived'])

SelectKBest(k='all', score_func=<function f_classif at 0x7f66721bbea0>)

In [41]:
feat_scores = pd.DataFrame()
feat_scores["F Score"] = feat_selector.scores_
feat_scores["P Value"] = feat_selector.pvalues_
feat_scores["Support"] = feat_selector.get_support()
feat_scores["Attribute"] = df.drop('Survived', axis=1).columns

In [43]:
feat_scores.sort_values(["F Score", "P Value"], ascending=[False, False])

,F Score,P Value,Support,Attribute
4,372.405724,1.406066e-69,True,Sex_male
8,103.057599,5.510281e-23,True,Pclass_3
3,63.030764,6.120189e-15,True,Fare
6,22.075469,3.036111e-06,True,Embarked_S
7,7.814805,5.293655e-03,True,Pclass_2
2,5.963464,1.479925e-02,True,Parch
0,4.353516,3.721708e-02,True,Age
1,1.110572,2.922439e-01,True,SibSp
5,0.011846,9.133532e-01,True,Embarked_Q


## 3. Eliminación recursiva de características

En Scikit Learn también vamos a encontrar una clase para realizar una eliminación recursiva de características. La misma se llama "RFECV". Usémosla en combinación de un modelo de regresión logística para ver qué características serán conservadas con este método.

Guardémoslas en una variable llamada "rfecv_columns"

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle = True)
estimator = SVR(kernel='linear')
selector = RFECV(estimator, step = 1, scoring = 'neg_mean_squared_error', verbose=2)
selector.fit(df2.drop('price',axis = 1),df2['price'])

## 4. Coeficientes de Regresión Logística

Veamos si los coeficientes de una RL se condicen.

- Creá un modelo de regresión logística
- Ejecutá un grid search sobre los parámetros "penalty type" y "C strength" para encontrar la mejor combinación
- Ordená los coeficientes obtenidos por valor absoluto (módulo). El top 5 coincide con los de arriba? Por qué/Por qué no? (Pista: Están todos los valores en la misma escala?)

Guardemos las que querramos mantener en una variable llamada "lr_columns"

## 5. Comparar sets de características

Usá el "best estimator" del punto anterior sobre los sets de características obtenidos:

- "kbest_columns"
- "rfecv_columns"
- "lr_columns"
- "all_columns"

Usá validación cruzada (cross_val_score) para evaluar los modelos 
Preguntas:

- Cuál obtuvo mejores resultados?
- Hay diferencias signigicativas?
- Cuál es la mejor opción? Por qué?

## Bonus

Creá un gráfico de barras para mostrar los coeficientes de la regresión logística.